In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(7819)

df = pd.read_csv('Education.csv', encoding='latin-1')
df = df[df['Attribute'] == 'Percent of adults with a bachelor\'s degree or higher 2007-11']
ozone = pd.read_csv('smallozone.csv')
ozone = ozone.groupby(['countyfips']).mean()

def hypothesis_test(start, end, df, ozone):
    state_df = df[(df['Federal Information Processing Standard (FIPS) Code'] > start) & (df['Federal Information Processing Standard (FIPS) Code'] < end)]
    state_df["Highly Educated"] = state_df['Value'] > 20
    joined = state_df.join(ozone, on='Federal Information Processing Standard (FIPS) Code', how="inner")
    means_table = joined.groupby(['Highly Educated']).mean()
    vals = means_table["ds_o3_pred"].tolist()
    difference = abs(vals[0] - vals[1])
    differences = []
    reps = 5000
    for i in range(reps):
        diff = one_simulation(joined)
        differences.append(diff)
    p_value = np.count_nonzero(np.array(differences) <= difference) / reps
    return p_value

def difference_of_means(table, group_label):
    reduced = table[['ds_o3_pred', group_label]]
    means_table = reduced.groupby(group_label).mean()
    means = means_table['ds_o3_pred']
    return abs(means[1] - means[0])

def one_simulation(table):
    shuffle = table['Highly Educated'].tolist()
    np.random.shuffle(shuffle)
    table['Shuffled Label'] =  shuffle
    tablecopy = table.copy(deep=True)
    return difference_of_means(tablecopy, 'Shuffled Label')

starts = list(np.arange(1000, 57000, 1000))
ends = list(np.arange(2000, 58000, 1000))
removes = [2000, 3000, 7000, 9000,10000, 11000, 14000, 15000,25000, 43000 , 44000, 52000]
for rem in removes:
    starts.remove(rem)
    ends.remove(rem + 1000)
p_values = []
for i in range(len(starts)):
    p_values.append(hypothesis_test(starts[i], ends[i], df.copy(deep=True), ozone.copy(deep=True)))


p_values.sort()
k = np.arange(1, 45)
line = [i * 0.05 / 44 for i in k]
plt.plot(k, line, label="k * a/m")
plt.plot(k, p_values, label = "p-values")
plt.legend()
plt.show()


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 1: invalid continuation byte